In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[166]:


get_ipython().run_line_magic('matplotlib', 'inline')
import numpy as np
import pandas as pd
import matplotlib.pylab
import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 20, 16


# In[167]:


import warnings
import itertools
warnings.filterwarnings("ignore") 


# In[170]:


df = pd.read_csv(r"C:\Users\USER\Documents\카카오톡 받은 파일\1년치 시계열데이터.csv")
df.plot()


# In[186]:


df.info()


# In[187]:


dateparse = lambda x: pd.to_datetime(x, errors = 'coerce')
df = pd.read_csv((r"C:\Users\USER\Documents\카카오톡 받은 파일\1년치 시계열데이터.csv"), parse_dates=['when'], index_col='when', date_parser=dateparse) 
df.head()


# In[188]:


ts = df[pd.Series(pd.to_datetime(df.index, errors='coerce')).notnull().values]
ts.head(15)


# In[189]:


ts.dtypes


# In[204]:


import statsmodels
import statsmodels.api as sm
from statsmodels.tsa.stattools import coint, adfuller


# In[205]:


def TestStationaryPlot(ts):
    rol_mean = ts.rolling(window = 12, center = False).mean()
    rol_std = ts.rolling(window = 12, center = False).std()
    
    plt.plot(ts, color = 'blue',label = 'Original Data')
    plt.plot(rol_mean, color = 'red', label = 'Rolling Mean')
    plt.plot(rol_std, color ='black', label = 'Rolling Std')
    plt.xlabel('Time in Years', fontsize = 5)
    plt.title('Rolling Mean & Standard Deviation', fontsize = 5)
    plt.show(block= True)


# In[206]:


def TestStationaryAdfuller(ts, cutoff = 0.01):
    data_test = adfuller(ts, autolag = 'AIC')
    data_test_output = pd.Series(data_test[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    
    for key,value in data_test[4].items():
        data_test_output['Critical Value (%s)'%key] = value
    print(data_test_output)
    
    if data_test[1] <= cutoff:
        print("Strong evidence against the null hypothesis, reject the null hypothesis. Data has no unit root, hence it is stationary")
    else:
        print("Weak evidence against null hypothesis, time series has a unit root, indicating it is non-stationary ")


# In[207]:


TestStationaryPlot(ts)


# In[214]:


TestStationaryAdfuller(ts['total'])


# In[221]:


moving_avg = ts.rolling(12).mean()
plt.plot(ts)
plt.plot(moving_avg, color='red')
plt.title('total number of delivery by series analysis', fontsize = 5)
plt.show()


# In[225]:


ts_moving_avg_diff = ts - moving_avg
ts_moving_avg_diff.head(13)


# In[226]:


ts_moving_avg_diff.dropna(inplace=True)
TestStationaryPlot(ts_moving_avg_diff)


# In[230]:


TestStationaryAdfuller(ts_moving_avg_diff['total'])


# In[ ]:


#The rolling mean values appear to be varying slightly. The Test Statistic is smaller than the 10% 5%, and 1% of critical values. So, we can say with 99% confidence level that the dataset is a stationary series.


# In[231]:


ts_first_difference = ts - ts.shift(11)  
TestStationaryPlot(ts_first_difference.dropna(inplace=False))
# Eliminating trend and seasonality: Differencing


# In[234]:


TestStationaryAdfuller(ts_first_difference['total'].dropna(inplace=False))


# In[236]:


ts_seasonal_difference = ts - ts.shift(12)  
TestStationaryPlot(ts_seasonal_difference['total'].dropna(inplace=False))
TestStationaryAdfuller(ts_seasonal_difference['total'].dropna(inplace=False))


# In[237]:


ts_seasonal_first_difference = ts_first_difference - ts_first_difference.shift(12)  
TestStationaryPlot(ts_seasonal_first_difference.dropna(inplace=False))


# In[240]:


TestStationaryAdfuller(ts_seasonal_first_difference['total'].dropna(inplace=False))


# In[245]:


from statsmodels.tsa.seasonal import seasonal_decompose
decomposition = seasonal_decompose(ts)

trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

plt.subplot(411)
plt.plot(ts, label='Original')
plt.legend(loc='best')
plt.subplot(412)
plt.plot(trend, label='Trend')
plt.legend(loc='best')
plt.subplot(413)
plt.plot(seasonal,label='Seasonality')
plt.legend(loc='best')
plt.subplot(414)
plt.plot(residual, label='Residuals')
plt.legend(loc='best')
plt.tight_layout()


# In[252]:


ts_decompose = residual
ts_decompose.dropna(inplace=True)
TestStationaryPlot(ts_decompose)
TestStationaryAdfuller(ts_decompose['total'])


# In[246]:


fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(ts_seasonal_first_difference.iloc[13:], lags=40, ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(ts_seasonal_first_difference.iloc[13:], lags=40, ax=ax2)


# In[263]:


p = d = q = range(0, 2) # Define the p, d and q parameters to take any value between 0 and 2
pdq = list(itertools.product(p, d, q)) # Generate all different combinations of p, q and q triplets
pdq_x_QDQs = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))] # Generate all different combinations of seasonal p, q and q triplets
print('Examples of Seasonal ARIMA parameter combinations for Seasonal ARIMA...')
print('SARIMAX: {} x {}'.format(pdq[1], pdq_x_QDQs[1]))
print('SARIMAX: {} x {}'.format(pdq[2], pdq_x_QDQs[2]))


# In[264]:


aic_results = []
for param in pdq:
    for seasonal_param in pdq_x_QDQs:
        try:
            mod = sm.tsa.statespace.SARIMAX(ts,
                                            order=param,
                                            seasonal_order=seasonal_param,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)
            results = mod.fit()
            print('ARIMA{}x{} - AIC:{}'.format(param, seasonal_param, results.aic))
            if results.mle_retvals is not None and results.mle_retvals['converged'] == False:
                print(results.mle_retvals)
            aic_results.append(results.aic)
        except:
            continue
aic_results.sort()
print('Best AIC found: ', aic_results[0])


# In[267]:


mod = sm.tsa.statespace.SARIMAX(ts, 
                                order=(1,0,1), 
                                seasonal_order=(1,1,1,12),   
                                enforce_stationarity=False,
                                enforce_invertibility=False)
results = mod.fit()
print(results.summary())


# In[268]:


results.resid.plot()


# In[269]:


print(results.resid.describe())


# In[270]:


results.resid.plot(kind='kde')


# In[271]:


results.plot_diagnostics(figsize=(15, 12))
plt.show()


# In[289]:


pred = results.get_prediction(start = 365, end = 400, dynamic=False)
pred_ci = pred.conf_int()
pred_ci.head()


# In[334]:


ax = ts['2019-10-01':].plot(label='observed')
pred.predicted_mean.plot(ax=ax, label='One-step ahead forecast', alpha=.7)

ax.fill_between(pred_ci.index,
                pred_ci.iloc[:, 0],
                pred_ci.iloc[:, 1], alpha=.5)

ax.set_xlabel('Time (years)')
ax.set_ylabel('total')
plt.legend()

plt.show()


# In[330]:


ts_forecast = pred.predicted_mean
ts_orginal = ts['2018-10-01':]

# Compute the mean square error
mse = ((ts_forecast - ts_orginal) ** 2).mean()
print('The Mean Squared Error (MSE) of the forecast is {}'.format(round(mse, 2)))
print('The Root Mean Square Error (RMSE) of the forcast: {:.4f}'
      .format(np.sqrt(sum((ts_forecast-ts_orginal)**2)/len(ts_forecast))))


# In[335]:


ts_pred_concat = pd.concat([ts_truth, ts_forecast])


# In[336]:


pred_dynamic = results.get_prediction(start=pd.to_datetime('2019-10-01'), dynamic=True, full_results=True)
pred_dynamic_ci = pred_dynamic.conf_int()


# In[356]:


ax = ts['2018-10-01':].plot(label='observed', figsize=(20, 15))
pred_dynamic.predicted_mean.plot(label='Dynamic Forecast', ax=ax)

ax.fill_between(pred_dynamic_ci.index,
                pred_dynamic_ci.iloc[:, 0],
                pred_dynamic_ci.iloc[:, 1], 
                color='r', alpha=.3)

ax.fill_betweenx(ax.get_ylim(), 
                 pd.to_datetime('2019-10-01'), 
                 mte.index[-1],
                 alpha=.1, zorder=-1)

ax.set_xlabel('Time (years)')
ax.set_ylabel('total')

plt.legend()
plt.show()


# In[357]:


ts_forecast = pred_dynamic.predicted_mean
ts_orginal = ts['2019-10-01':]

# Compute the mean square error
mse = ((ts_forecast - ts_orginal) ** 2).mean()
print('The Mean Squared Error (MSE) of the forecast is {}'.format(round(mse, 2)))
print('The Root Mean Square Error (RMSE) of the forcast: {:.4f}'
      .format(np.sqrt(sum((ts_forecast-ts_orginal)**2)/len(ts_forecast))))


# In[358]:


forecast = results.get_forecast(steps= 40)
# Get confidence intervals of forecasts
forecast_ci = forecast.conf_int()
forecast_ci.head()


# In[359]:


ax = ts.plot(label='observed', figsize=(20, 15))
forecast.predicted_mean.plot(ax=ax, label='Forecast')
ax.fill_between(forecast_ci.index,
                forecast_ci.iloc[:, 0],
                forecast_ci.iloc[:, 1], color='g', alpha=.4)
ax.set_xlabel('Time (year)')
ax.set_ylabel('total')

plt.legend()
plt.show()



FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\USER\\Documents\\카카오톡 받은 파일\\1년치 시계열데이터.csv'